# **Aula 08 - Introdução a Machine Learning para PLN**

**Aluna:** Giovanna da Rocha Machado | **RA:** 1131392213024

Nesta aula, exploramos conceitos básicos de Machine Learning aplicado ao Processamento de Linguagem Natural (PLN).
Abordaremos classificação de sentimentos com vetorização de texto com TF-IDF, modelos de aprendizado supervisionado.

A aula inclui dois exemplos práticos:

- Um estudo com um corpus simples de frases em português para treinar e avaliar um modelo SVM.
- Um exemplo mais avançado com o dataset com análises de filmes do NLTK, onde compararemos o desempenho dos modelos Naive Bayes e SVM em uma tarefa de classificação binária.

## **Exemplo 01**: Aplicação do modelo de Naives em um texto

In [8]:
# Passo 1 - Criar o Corpus

corpus = [
    ("Eu amo PLN", "positivo"),
    ("Eu odeio bugs", "negativo"),
    ("Amo resolver problemas", "positivo"),
    ("Odeio erros", "negativo"),
    ("Bugs são legais", "positivo"),
    ("Resolver dificuldades é vida", "positivo"),
    ("Eu não gosto de falhas", "negativo"),
]

# Passo 2 - Processar o Texto
import re # Lib para trabalhar com expressões regulares
from collections import defaultdict, Counter # Contagem e estruturação de dados

def preprocess_text(text):
  return re.findall(r'\b\w+\b', text.lower())

# Converte o texto para minúsculas e divide em tokens utilizando expressões regulares.
processed_corpus = [(preprocess_text(text), label) for text, label in corpus]
print(processed_corpus)

# Passo 3 - Calculando probabilidades
class_counts = Counter() # Contador para armazenar a quantidade de exemplos em cada classe
word_counts = defaultdict(Counter)  # Armazenar a contagem de palavras em cada classe
total_words = defaultdict(int) # Armazenar o total de palavras em cada classe

# Itera sobre o corpus pré-processado para calcular as contagens
for words, label in processed_corpus:
  class_counts[label] += 1 # Incrementa a contagem de exemplos para a classe
  for word in words:
    word_counts[label][word] += 1  # Incrementa a contagem da palavra na classe
    total_words[label]

# Nº total de exemplos no corpus
total_examples = sum(class_counts.values())

# Calculando as probabilidades a priori (P(classe)) para cada classe
prior_probabilities = {cls: count / total_examples for cls, count in class_counts.items()}

# Função para calcular a probabilidade condicional P(palavra|classe) com suavização de Laplace (alpha)
def conditional_probability(word, label, alpha=1):
  # Fórmula: (contagem da palavra + alpha) / (total de palavras na classe + alpha * tamanho do vocabulário)
  return (word_counts[label][word] + alpha) / (total_words[label] + alpha * len(word_counts[label]))

# Passo 4 - Classificar um novo texto
def predict(text):
  words = preprocess_text(text) # Pré-processa o texto de entrada (tokenização)
  probabilities = {} # Armazenar as probabilidades para cada classe

  # Calculando as probabilidades para cada classe
  for label in class_counts.keys():
    probabilities[label] = prior_probabilities[label]  # Inicializa com a probabilidade a priori da classe
    for word in words:
      probabilities[label] *= conditional_probability(word, label) # Multiplica pela probabilidade condicional
  return max(probabilities, key=probabilities.get), probabilities # Retorna a classe com maior probabilidade

# Passo 5 - Teste com um novo texto
novo_texto = "Eu amo resolver bugs"

# Previsão utilizando a função predict
classe, probs = predict(novo_texto)

print(f'Texto: "{novo_texto}"')
print(f'Classe prevista: "{classe}"')
print(f'Probabilidades:')
for label, prob in probs.items():
  print(f'{label}"{prob}"')

[(['eu', 'amo', 'pln'], 'positivo'), (['eu', 'odeio', 'bugs'], 'negativo'), (['amo', 'resolver', 'problemas'], 'positivo'), (['odeio', 'erros'], 'negativo'), (['bugs', 'são', 'legais'], 'positivo'), (['resolver', 'dificuldades', 'é', 'vida'], 'positivo'), (['eu', 'não', 'gosto', 'de', 'falhas'], 'negativo')]
Texto: "Eu amo resolver bugs"
Classe prevista: "positivo"
Probabilidades:
positivo"0.0014050562510367166"
negativo"0.0006277901785714285"


## **Exemplo 2**: Classificação de Sentimentos com SVM e TF-IDF

In [9]:
!pip install scikit-learn

In [10]:
# Passo 1: Importação das bibliotecas necessárias
from sklearn.feature_extraction.text import TfidfVectorizer  # Para representar texto com TF-IDF
from sklearn.svm import SVC  # Para o modelo de máquina de vetores de suporte (SVM)
from sklearn.model_selection import train_test_split  # Para dividir os dados em treino e teste
from sklearn.metrics import classification_report  # Para avaliar a performance do modelo

# Passo 2: Definição do corpus e classes
corpus = [
    "Eu amo PLN",
    "Eu odeio bugs",
    "Eu amo resolver problemas",
    "Odeio erros",
    "Amo programação",
    "Não gosto de falhas"
]
classes = ["positivo", "negativo", "positivo", "negativo", "positivo", "negativo"] # Cada texto no corpus possui uma classe associada

# Passo 3: Pré-processamento e vetorização com TF-IDF
vectorizer = TfidfVectorizer()  # Instancia o vetor TF-IDF
X = vectorizer.fit_transform(corpus)  # Ajusta e transforma o corpus em uma matriz numérica
y = classes  # As classes alvo

# Passo 4: Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# test_size=0.3 significa que 30% dos dados serão usados para teste
# random_state=42 garante reprodutibilidade

# Treinamento do modelo SVM
svm_model = SVC(kernel='linear')  # Cria um modelo SVM com kernel linear
svm_model.fit(X_train, y_train)  # Treinar o modelo com os dados de treino

# Passo 5: Avaliação do modelo
y_pred = svm_model.predict(X_test)  # Faz predições no conjunto de teste
print(classification_report(y_test, y_pred))  # Exibe métricas de desempenho


              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00         1
    positivo       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



## **Exemplo 3**: Classificação de Sentimentos em Inglês com TF-IDF, Naive Bayes e SVM

In [11]:
# 1. Importação das bibliotecas necessárias
import nltk  # Biblioteca para processamento de linguagem natural
from sklearn.feature_extraction.text import TfidfVectorizer  # Vetorização com TF-IDF
from sklearn.model_selection import train_test_split  # Divisão dos dados em conjuntos de treino e teste
from sklearn.naive_bayes import MultinomialNB  # Modelo de classificação baseado no Naive Bayes
from sklearn.svm import SVC  # SVM
from sklearn.metrics import classification_report  # Métricas de avaliação como precisão, recall e F1-score
from sklearn.preprocessing import LabelEncoder  # Codificação de labels em valores numéricos

# Baixar o dataset de exemplo
nltk.download('movie_reviews')  # Baixa o corpus de análises de filmes
from nltk.corpus import movie_reviews  # Importando corpus pós download

# 2. Preparação dos dados
# Coleta os textos e suas classes (positiva / negativa)
documents = [
    (" ".join(movie_reviews.words(fileid)), category)  # Junta palavras de cada análise em uma string
    for category in movie_reviews.categories()  # Itera sobre categorias 'pos' e 'neg'
    for fileid in movie_reviews.fileids(category)  # Itera nos IDs de arquivo para cada categoria
]
# O corpus resulta em uma lista de tuplas (texto, classe)

# Separar textos e rótulos em variáveis separadas
texts, labels = zip(*documents)  # 'texts' contém análises, 'labels' contém classes

# Transformar rótulos em valores numéricos (0 = 'neg', 1 = 'pos')
label_encoder = LabelEncoder()  # Codificador de rótulos
labels = label_encoder.fit_transform(labels)  # Converte classes string para números

# Divide os dados em conjuntos de treino e teste
texts_train, texts_test, labels_train, labels_test = train_test_split(
    texts,  # Textos
    labels,  # Classes
    test_size=0.3,  # 30% dos dados para o conjunto de teste
    random_state=42  # Semente aleatória
)

# 3. Representação do texto com TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # max_features: Limita o vocabulário às x palavras mais frequentes
X_train = vectorizer.fit_transform(texts_train)  # Ajusta o vetorizador TF-IDF aos textos de treino e transforma em vetores numéricos
X_test = vectorizer.transform(texts_test)  # Transforma os textos teste em vetores numéricos usando o modelo ajustado no treino

# 4. Treinamento dos modelos

# Treinamento do Naive Bayes
nb_model = MultinomialNB()  # Modelo Naive Bayes Multinomial, adequado para dados de contagem (como TF-IDF)
nb_model.fit(X_train, labels_train)  # Treina o modelo com os vetores TF-IDF de treino e rótulos

# Predição com Naive Bayes
nb_predictions = nb_model.predict(X_test)  # Usa o modelo treinado para prever classes dos textos de teste

# Treinamento do SVM
svm_model = SVC(kernel='linear')
 # SVC: Modelo de Máquina de Vetores de Suporte
# kernel='linear': Utiliza kernel linear para encontrar hiperplanos que separam as classes

svm_model.fit(X_train, labels_train)  # Treina o modelo SVM com os vetores TF-IDF de treino e rótulos

# Predição com SVM
svm_predictions = svm_model.predict(X_test)  # Usa o modelo treinado para prever as classes dos textos de teste

# 5. Avaliação dos modelos

# Avaliação do Naive Bayes
print("Desempenho do Naive Bayes:")
print(classification_report(labels_test, nb_predictions, target_names=label_encoder.classes_))
# classification_report: Métricas como precisão, recall, F1-score...
# labels_test: Rótulos reais
# nb_predictions: Rótulos previstos pelo modelo Naive Bayes
# target_names: Nomes originais classes ('neg', 'pos')

# Avaliação do SVM
print("Desempenho do SVM:")
print(classification_report(labels_test, svm_predictions, target_names=label_encoder.classes_))
# labels_test: Rótulos reais
# svm_predictions: Rótulos previstos pelo modelo SVM
# target_names: Nomes originais classes ('neg', 'pos')


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


Desempenho do Naive Bayes:
              precision    recall  f1-score   support

         neg       0.79      0.84      0.81       302
         pos       0.82      0.77      0.80       298

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600

Desempenho do SVM:
              precision    recall  f1-score   support

         neg       0.82      0.80      0.81       302
         pos       0.81      0.82      0.81       298

    accuracy                           0.81       600
   macro avg       0.81      0.81      0.81       600
weighted avg       0.81      0.81      0.81       600

